In [ ]:
# !pip install openai faker sqlalchemy snowflake-connector-python pandas snowflake-sqlalchemy openai.error RateLimitError

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import uuid
from typing import Dict, List
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from snowflake.sqlalchemy import URL as SnowflakeURL
from snowflake.connector.pandas_tools import write_pandas
from openai import OpenAI
import json
from faker import Faker
import os
from pathlib import Path
import logging
import time
import sys

In [ ]:
class TravelGenieReviewGenerator:
    def __init__(self, engine, openai_api_key: str):
        self.engine = engine
        self.openai_client = OpenAI(api_key=openai_api_key)
        self.fake = Faker()
        self.load_reference_data()
        logging.basicConfig(
            level=logging.INFO,
            filename='generator.log',
            filemode='a',
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

    def load_reference_data(self):
        """Load existing reviews to determine synthetic data volume"""
        # Get count of existing reviews per attraction
        self.review_counts = pd.read_sql("""
            SELECT
                LOCATIONID,
                COUNT(*) as review_count
            FROM TRAVEL_GENIE.RAW.ATTRACTION_REVIEWS
            GROUP BY LOCATIONID
        """, self.engine)
        print("Columns in self.review_counts:", self.review_counts.columns.tolist())

        # Load attraction data
        self.attractions_df = pd.read_sql("""
            SELECT
                ID, NAME, SUBCATEGORIES, LOCATION_STRING,
                LATITUDE, LONGITUDE, DESCRIPTION
            FROM TRAVEL_GENIE.RAW.ATTRACTIONS
        """, self.engine)
        print("Columns in self.attractions_df:", self.attractions_df.columns.tolist())

        # Load sample reviews for patterns
        self.sample_reviews = pd.read_sql("""
            SELECT
                LOCATIONID, RATING, TEXT, TITLE, TRIP_TYPE,
                TRAVELDATE, PUBLISHEDDATE
            FROM TRAVEL_GENIE.RAW.ATTRACTION_REVIEWS
            ORDER BY PUBLISHEDDATE DESC
            LIMIT 1000
        """, self.engine)
        print("Columns in self.sample_reviews:", self.sample_reviews.columns.tolist())

    def calculate_synthetic_reviews_needed(self) -> Dict[int, int]:
        """Calculate number of synthetic reviews needed per attraction for 70/30 split"""
        synthetic_counts = {}
        if 'review_count' in self.review_counts.columns:
            for _, row in self.review_counts.iterrows():
                current_reviews = row['review_count']
                # Calculate how many reviews we need for the current reviews to be 70%
                total_reviews_needed = int(current_reviews / 0.99)  # This gives us the 100% number
                synthetic_needed = total_reviews_needed - current_reviews  # This gives us the 30%

                logging.info(f"""
                    Attraction ID: {row['locationid']}
                    Current reviews: {current_reviews}
                    Total needed for 99/01 split: {total_reviews_needed}
                    Synthetic reviews needed: {synthetic_needed}
                    Final split will be: {current_reviews}/{synthetic_needed}
                    ({(current_reviews/(current_reviews + synthetic_needed))*100:.1f}% /
                    {(synthetic_needed/(current_reviews + synthetic_needed))*100:.1f}%)
                """)

                synthetic_counts[row['locationid']] = max(1, synthetic_needed)
        else:
            raise ValueError("Column 'review_count' is missing in the loaded data.")
        return synthetic_counts

    def _create_review_prompt(self, attraction: pd.Series) -> str:
      """Create enhanced prompt for GPT with natural variations in length, sentiment, and detail"""

    # Review style variations
      review_styles = [
          {"style": "detailed", "length": "long", "focus": "comprehensive analysis", "word_count": "400-600 words"},
          {"style": "concise", "length": "short", "focus": "key highlights", "word_count": "100-200 words"},
          {"style": "balanced", "length": "medium", "focus": "main points with detail", "word_count": "250-350 words"},
          {"style": "emotional", "length": "medium", "focus": "personal experience", "word_count": "250-350 words"},
          {"style": "practical", "length": "medium", "focus": "tips and advice", "word_count": "250-350 words"}
      ]

    # Different aspects that visitors might focus on
      focus_aspects = [
          "value for money and costs",
          "crowd levels and best times to visit",
          "cleanliness and maintenance",
          "staff service and helpfulness",
          "facilities and amenities",
          "accessibility and convenience",
          "authenticity and cultural aspects",
          "atmosphere and overall experience",
          "safety and security measures",
          "photo opportunities and scenery",
          "food and beverage options",
          "parking and transportation",
          "restroom availability and condition",
          "signage and wayfinding",
          "special features or unique offerings"
      ]

    # Different visitor perspectives with expectations
      visitor_types = [
          {"type": "budget traveler", "expectations": "value, affordable options, cost-effectiveness"},
          {"type": "luxury seeker", "expectations": "premium experience, quality service, exclusivity"},
          {"type": "family visitor", "expectations": "child-friendly, safety, entertainment"},
          {"type": "photography enthusiast", "expectations": "photo spots, lighting, views"},
          {"type": "history buff", "expectations": "historical accuracy, information, preservation"},
          {"type": "adventure seeker", "expectations": "excitement, unique experiences, activities"},
          {"type": "relaxation focused", "expectations": "comfort, peaceful atmosphere, amenities"},
          {"type": "cultural explorer", "expectations": "authenticity, local insights, traditions"},
          {"type": "accessibility concerned", "expectations": "facilities, movement ease, support"},
          {"type": "time-constrained visitor", "expectations": "efficiency, highlights, organization"}
     ]

    # Randomly select elements for this review
      selected_style = random.choice(review_styles)
      selected_aspects = random.sample(focus_aspects, k=random.randint(2, 4))
      selected_visitor = random.choice(visitor_types)

    # Calculate average rating from sample reviews for this attraction
      attraction_reviews = self.sample_reviews[
        self.sample_reviews['locationid'] == attraction['id']
    ]
      if not attraction_reviews.empty:
          avg_rating = attraction_reviews['rating'].mean()
          rating_distribution = attraction_reviews['rating'].value_counts(normalize=True)
          rating_context = f"\nTypical ratings for this attraction range around {avg_rating:.1f} stars."
      else:
          rating_context = "\nProvide a rating based on the authentic experience described."

      return f"""Generate a {selected_style['length']} TripAdvisor review ({selected_style['word_count']}) for:

ATTRACTION:
Name: {attraction['name']}
Location: {attraction['location_string']}
Categories: {attraction['subcategories']}
Description: {attraction['description']}

REVIEW PARAMETERS:
Style: {selected_style['style']} review focusing on {selected_style['focus']}
Perspective: Writing as a {selected_visitor['type']} who expects {selected_visitor['expectations']}
Key Aspects to Address: {', '.join(selected_aspects)}

REVIEW REQUIREMENTS:
1. Match the specified {selected_style['length']} length ({selected_style['word_count']})
2. Write from the {selected_visitor['type']} perspective
3. Focus especially on: {', '.join(selected_aspects)}
4. Include specific examples and personal observations
5. Mention both positives and negatives as appropriate
6. Add practical tips or recommendations
7. Include unique observations or unexpected findings
8. Use natural, conversational language
9. Consider seasonal or timing factors if relevant

RATING GUIDANCE:
- Rate based on the actual experience described
- Consider both positive and negative aspects
- Not every experience needs to be 5-star
- Mix of positives and negatives can justify 3-4 stars
- Significant issues can warrant 1-2 stars
- Consider value received vs expectations{rating_context}

Return ONLY a JSON object in this format:
{{
    "rating": <integer 1-5, matching the review content>,
    "title": "<brief, engaging title reflecting the main point>",
    "review_text": "<{selected_style['length']} review matching style and focus>",
    "trip_type": "<FAMILY|COUPLES|SOLO|BUSINESS|FRIENDS>",
    "travel_date": "<YYYY-MM-DD between 2021-2024>"
}}"""
    def generate_review(self, attraction_id):
        """Generate a single synthetic review using OpenAI API with handling for review variation"""
        attraction = self.attractions_df[self.attractions_df['id'] == attraction_id]
        if attraction.empty:
            logging.error(f"Attraction ID {attraction_id} not found in attractions DataFrame.")
            raise ValueError(f"Attraction ID {attraction_id} not found.")

        attraction = attraction.iloc[0]
        prompt = self._create_review_prompt(attraction)

        for attempt in range(3):
            try:
                response = self.openai_client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {
                            "role": "system",
                            "content": """You are a review generator that creates authentic, varied reviews.
                            Each review should have its own unique voice and perspective.
                            Match the requested length and style while maintaining natural language.
                            Consider both positive and negative aspects based on the specific experience.
                            Output only valid JSON."""
                        },
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.35,
                    max_tokens=800
                )

                review_text = response.choices[0].message.content.strip()
                logging.info(f"Raw GPT response for attraction {attraction_id}: {review_text}")

                try:
                    review_data = json.loads(review_text)
                except json.JSONDecodeError as je:
                    logging.error(f"Invalid JSON received for attraction {attraction_id}: {review_text}")
                    raise je

                required_fields = ['rating', 'title', 'review_text', 'trip_type', 'travel_date']
                missing_fields = [field for field in required_fields if field not in review_data]
                if missing_fields:
                    raise ValueError(f"Missing required fields: {missing_fields}")

                logging.info(f"""
                    Review generated for attraction {attraction_id}:
                    Length: {len(review_data['review_text'])} chars
                    Rating: {review_data['rating']}
                    Trip Type: {review_data['trip_type']}
                """)

                review = {
                    'id': str(uuid.uuid4()),
                    'locationid': attraction_id,
                    'rating': review_data['rating'],
                    'title': review_data['title'],
                    'text': review_data['review_text'],
                    'trip_type': review_data['trip_type'],
                    'traveldate': review_data['travel_date'],
                    'publisheddate': datetime.now().strftime('%Y-%m-%d'),
                    'username': self.fake.user_name(),
                    'userlocation': self.fake.city() + ', ' + self.fake.country()
                }
                return review

            except json.JSONDecodeError as e:
                logging.error(f"JSON decode error for attraction {attraction_id}: {str(e)}")
                time.sleep(2)
                continue
            except Exception as e:
                error_message = str(e)
                logging.error(f"Error generating review for attraction {attraction_id}: {error_message}")
                if "rate limit" in error_message.lower():
                    logging.info("Rate limit exceeded. Retrying after 10 seconds...")
                    time.sleep(10)
                    continue
                else:
                    logging.error(f"Unexpected error for attraction {attraction_id}: {error_message}")
                    break

        raise Exception(f"Failed to generate review for attraction {attraction_id} after multiple attempts.")

    def generate_synthetic_dataset(self):
        """Generate appropriate number of synthetic reviews"""
        synthetic_counts = self.calculate_synthetic_reviews_needed()
        synthetic_reviews = []

        valid_attraction_ids = set(self.attractions_df['id'])
        synthetic_counts = {aid: count for aid, count in synthetic_counts.items()
                          if aid in valid_attraction_ids}

        for attraction_id, num_reviews in synthetic_counts.items():
            print(f"Generating {num_reviews} reviews for attraction {attraction_id}")
            for _ in range(num_reviews):
                try:
                    review = self.generate_review(attraction_id)
                    synthetic_reviews.append(review)
                    time.sleep(0.5)
                except Exception as e:
                    logging.error(f"Error generating review for attraction {attraction_id}: {str(e)}")
                    continue

        return pd.DataFrame(synthetic_reviews)

    def save_to_snowflake(self, df: pd.DataFrame, table_name='ATTRACTION_REVIEWS_TEST'):
        """Save synthetic reviews to Snowflake with configurable table name"""
        try:
            conn = self.engine.raw_connection().connection

            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS TRAVEL_GENIE.RAW.{table_name} (
                ID VARCHAR(255),
                LOCATIONID NUMBER,
                RATING NUMBER,
                TITLE VARCHAR(1000),
                TEXT VARCHAR(16777216),
                TRIP_TYPE VARCHAR(50),
                TRAVELDATE DATE,
                PUBLISHEDDATE DATE,
                USERNAME VARCHAR(255),
                USERLOCATION VARCHAR(255)
            )
            """

            with conn.cursor() as cur:
                cur.execute(create_table_sql)

            df.columns = [col.upper() for col in df.columns]

            success, nchunks, nrows, _ = write_pandas(
                conn=conn,
                df=df,
                table_name=table_name,
                database='TRAVEL_GENIE',
                schema='RAW',
                quote_identifiers=False
            )
            print(f"Successfully added {nrows} synthetic reviews to {table_name}")
            return success
        except Exception as e:
            print(f"Error saving to Snowflake: {str(e)}")
            logging.error(f"Error saving to Snowflake: {str(e)}")
            return False

    def export_reviews(self, df: pd.DataFrame, output_dir: str = 'synthetic_reviews'):
        """Export reviews to files for inspection"""
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        csv_path = os.path.join(output_dir, 'synthetic_reviews.csv')
        df.to_csv(csv_path, index=False)
        print(f"Exported reviews to CSV: {csv_path}")

        json_path = os.path.join(output_dir, 'synthetic_reviews.json')
        df.to_json(json_path, orient='records', indent=2)
        print(f"Exported reviews to JSON: {json_path}")

        stats = {
            'total_reviews': len(df),
            'reviews_by_rating': df['rating'].value_counts().to_dict(),
            'reviews_by_trip_type': df['trip_type'].value_counts().to_dict(),
            'reviews_by_attraction': df['locationid'].value_counts().to_dict(),
            'date_range': {
                'earliest': df['traveldate'].min(),
                'latest': df['traveldate'].max()
            }
        }

        stats_path = os.path.join(output_dir, 'generation_statistics.json')
        with open(stats_path, 'w', encoding='utf-8') as f:
            json.dump(stats, f, indent=2, default=str)
        print(f"Exported statistics to: {stats_path}")

def main():
    conn_params = {
        'user': 'BULLFROG',
        'password': '9987323889Ritesh.',
        'account': 'SFEDU02-URB63596',
        'role': 'TRAINING_ROLE',
        'warehouse': 'ANIMAL_TASK_WH',
        'database': 'TRAVEL_GENIE',
        'schema': 'RAW',
        'authenticator': 'snowflake'
    }

    engine = create_engine(
        SnowflakeURL(
            user=conn_params['user'],
            password=conn_params['password'],
            account=conn_params['account'],
            role=conn_params['role'],
            warehouse=conn_params['warehouse'],
            database=conn_params['database'],
            schema=conn_params['schema'],
            authenticator=conn_params.get('authenticator', 'snowflake')
        )
    )

    generator = TravelGenieReviewGenerator(
        engine=engine,
        openai_api_key=''
    )

    try:
        print("Starting synthetic review generation...")
        synthetic_df = generator.generate_synthetic_dataset()

        if synthetic_df.empty:
            print("No synthetic reviews were generated.")
            return

        generator.export_reviews(synthetic_df)

        confirmation = input("Review the exported data. Save to Snowflake? (yes/no): ")

        if confirmation.lower() == 'yes':
            table_name = 'ATTRACTION_REVIEWS_TEST'
            success = generator.save_to_snowflake(synthetic_df, table_name)
            if success:
                print(f"Successfully saved synthetic reviews to {table_name}")
            else:
                print("Failed to save to Snowflake")
        else:
            print("Operation cancelled. Reviews were only exported to files.")

    finally:
        engine.dispose()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'faker'